<a href="https://colab.research.google.com/github/GerardoMunoz/AlgLin_2025/blob/main/cuestionarios/complexity_features_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import ast

In [29]:
a=ast.parse("a = 3*x + 1", mode="exec")
print(ast.dump(a))

Module(body=[Assign(targets=[Name(id='a', ctx=Store())], value=BinOp(left=BinOp(left=Constant(value=3), op=Mult(), right=Name(id='x', ctx=Load())), op=Add(), right=Constant(value=1)))], type_ignores=[])


In [30]:
import ast

def complexity_features_equation(eq_str, var="x"):
    """
    Calcula métricas de complejidad directamente de un string de ecuación,
    sin simplificar como SymPy.
    Retorna: (terms, parens, depth, repeat_penalty, dispersion)
    """
    if "=" not in eq_str:
        raise ValueError("Equation must contain '='")

    left_str, right_str = eq_str.split("=")

    def analyze(expr_str):
        tree = ast.parse(expr_str, mode='eval')

        terms = 0
        parens = expr_str.count("(") + expr_str.count(")")
        depth_val = 0
        var_count = expr_str.count(var)

        def visit(node, current_depth=1):
            nonlocal terms, depth_val
            depth_val = max(depth_val, current_depth)
            if isinstance(node, ast.BinOp):
                terms += 1
                visit(node.left, current_depth + 1)
                visit(node.right, current_depth + 1)
            elif isinstance(node, ast.UnaryOp):
                visit(node.operand, current_depth + 1)
            elif isinstance(node, ast.Call):
                terms += 1
                for arg in node.args:
                    visit(arg, current_depth + 1)
            elif isinstance(node, ast.Name) or isinstance(node, ast.Constant):
                pass
            elif hasattr(node, "body"):  # defensive
                for sub in node.body:
                    visit(sub, current_depth + 1)

        visit(tree.body)
        return terms, parens, depth_val, var_count

    left = analyze(left_str)
    right = analyze(right_str)

    # combine sides
    terms = left[0] + right[0]
    parens = left[1] + right[1]
    #depth = max(left[2], right[2])
    var_total = left[3] + right[3]
    repeat_penalty = var_total - 1 if var_total > 1 else 0
    #dispersion = (1 if (left[3] > 0 and right[3] > 0) else 0)

    return (terms, parens, repeat_penalty) # (terms, parens, depth, repeat_penalty, dispersion)


In [40]:
eq1 = "3*x+3*(x-2)-6*(x+(3-x))=6+2*x"
print(eq1, complexity_features_equation(eq1))

3*x+3*(x-2)-6*(x+(3-x))=6+2*x (10, 6, 4)


In [31]:
eq1 = "3*x+3*(x-2)-6*(x+(3-x))-2*x=6+2*x-2*x"
print(eq1, complexity_features_equation(eq1))

3*x+3*(x-2)-6*(x+(3-x))-2*x=6+2*x-2*x (14, 6, 6)


In [32]:
eq1 = "3*x+3*(x-2)-6*(x+(3-x))-2*x=6+0"
print(eq1, complexity_features_equation(eq1))

3*x+3*(x-2)-6*(x+(3-x))-2*x=6+0 (11, 6, 4)


In [33]:
eq1 = "3*x+3*(x-2)-6*(x+(3-x))-2*x=6"
print(eq1, complexity_features_equation(eq1))

3*x+3*(x-2)-6*(x+(3-x))-2*x=6 (10, 6, 4)


In [34]:
eq1 = "3*x+3*x-2*3-6*(x+(3-x))-2*x=6"
print(eq1, complexity_features_equation(eq1))

3*x+3*x-2*3-6*(x+(3-x))-2*x=6 (11, 4, 4)


In [35]:
eq1 = "6*x-2*3-6*(x+(3-x))-2*x=6"
print(eq1, complexity_features_equation(eq1))

6*x-2*3-6*(x+(3-x))-2*x=6 (9, 4, 3)


In [36]:
eq1 = "6*x-2*3-6*(x+3-x)-2*x=6"
print(eq1, complexity_features_equation(eq1))

6*x-2*3-6*(x+3-x)-2*x=6 (9, 2, 3)


In [37]:
eq1 = "6*x-2*3-6*x-18+6*x-2*x=6"
print(eq1, complexity_features_equation(eq1))

6*x-2*3-6*x-18+6*x-2*x=6 (10, 0, 3)


In [38]:
eq1 = "6*x-2*3-6*x-18+6*x-2*x=6"
print(eq1, complexity_features_equation(eq1))

6*x-2*3-6*x-18+6*x-2*x=6 (10, 0, 3)


In [39]:
eq1 = "6*x-2*3-6*x-18+6*x-2*x=6"
print(eq1, complexity_features_equation(eq1))

6*x-2*3-6*x-18+6*x-2*x=6 (10, 0, 3)
